In [1]:
#from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import pandas as pd
import numpy as np
import random
import tensorflow as tf

from gensim.models import Word2Vec

In [2]:
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding,Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential 
#from tensorflow.keras.models import Sequential, load_model
#from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding, Masking, Bidirectional
#from tensorflow.keras.optimizers import Adam
#from tensorflow.keras import backend

In [3]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import LancasterStemmer

nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
data = pd.read_csv("/content/drive/My Drive/NLP/Project/NLPMiniProject_data.csv")
print(data.head())

           ID                                            comment     date  \
0  uid_590555  Well, let's be honest here, they don't actuall...  2015-04   
1  uid_671762  Well, I didn't need evidence to believe in com...  2016-12   
2  uid_519689              Who does an "official promo" in 360p?  2013-11   
3  uid_788362                           Grotto koth was the best  2015-09   
4  uid_299252                                   Neal's back baby  2015-11   

   down                                     parent_comment  score  top  \
0     0  They should shut the fuck up and let the commu...      2    2   
1    -1  You need evidence to kill people? I thought we...      6   -1   
2     0                    2014 BMW S1000R: Official Promo      3    3   
3     0  Not really that memorable lol if you want memo...      2    2   
4     0                      James Neal hit on Zach Parise     -5   -5   

              topic            user  label  
0       starcitizen  Combat_Wombatz      0  
1 

In [6]:
df = data[['ID','comment','parent_comment','down','score','top','topic','label']]

In [7]:
from nltk.tokenize import RegexpTokenizer

In [8]:
tokenizer = RegexpTokenizer('\w+')

In [9]:
#Tokenization based on space
data_list1 = list()
for row in df['comment']:
    data_list1.append(RegexpTokenizer('\w+').tokenize(row))

In [10]:
#Tokenization based on space
data_list2 = list()
for row in df['parent_comment']:
    data_list2.append(RegexpTokenizer('\w+').tokenize(row))

In [11]:
#converting to lower case
data_list_lower1 = []
for i in data_list1:
    lower = list(map(lambda x:x.lower(),i))
    data_list_lower1.append(lower)   

In [12]:
#converting to lower case
data_list_lower2 = []
for i in data_list2:
    lower = list(map(lambda x:x.lower(),i))
    data_list_lower2.append(lower)   

In [13]:
#remove puncts
from string import punctuation

In [14]:
stop_words = list(stopwords.words('english'))

In [15]:
final_stop_words = list(punctuation) + stop_words

In [16]:
data_list_lower_stpwords1 = []
for line in data_list_lower1:
    stpwords = []
    for w in line:
        if w not in final_stop_words:
            stpwords.append(w)
    data_list_lower_stpwords1.append(stpwords)  

In [17]:
data_list_lower_stpwords2 = []
for line in data_list_lower2:
    stpwords = []
    for w in line:
        if w not in final_stop_words:
            stpwords.append(w)
    data_list_lower_stpwords2.append(stpwords)  

In [18]:
#Lemmatization
lemma = WordNetLemmatizer()
data_list_lower_stpwords_lem1 = []
for line in data_list_lower_stpwords1:
    lines = list(map(lambda x:lemma.lemmatize(x),line))
    data_list_lower_stpwords_lem1.append(lines)

In [19]:
#Lemmatization
lemma = WordNetLemmatizer()
data_list_lower_stpwords_lem2 = []
for line in data_list_lower_stpwords2:
    lines = list(map(lambda x:lemma.lemmatize(x),line))
    data_list_lower_stpwords_lem2.append(lines)

In [28]:
df['clean_comment_tokenized'] = data_list_lower_stpwords_lem1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [30]:
df['clean_parent_comment_tokenized'] = data_list_lower_stpwords_lem2

In [39]:
#using comment field alone

In [40]:
X_train, X_test, y_train, y_test = train_test_split(
    df['clean_comment_tokenized'],
    df['label'],
    test_size=0.2, 
    random_state=42
)

In [41]:
X_train.shape, X_test.shape

((12000,), (3000,))

In [42]:
X_train.reset_index(inplace=True, drop=True)

In [43]:
X_test.reset_index(inplace=True, drop=True)

In [44]:
desired_vocab_size = 1000 #Vocablury size
t = tf.keras.preprocessing.text.Tokenizer(num_words=desired_vocab_size) # num_words -> Vocablury size

In [46]:
#Fit tokenizer with actual training data
t.fit_on_texts(X_train.tolist())

In [47]:
#Vocabulary
print(t.word_index)

{'like': 1, 'get': 2, 'yeah': 3, 'would': 4, 'people': 5, 'one': 6, 'well': 7, 'know': 8, 'make': 9, 'good': 10, 'game': 11, 'right': 12, 'think': 13, 'time': 14, 'really': 15, 'need': 16, 'sure': 17, 'thing': 18, 'yes': 19, 'guy': 20, 'mean': 21, 'u': 22, 'even': 23, 'want': 24, 'go': 25, 'man': 26, 'see': 27, 'much': 28, 'never': 29, 'way': 30, 'oh': 31, 'look': 32, 'say': 33, 'could': 34, 'better': 35, 'going': 36, 'got': 37, 'still': 38, 'back': 39, 'year': 40, 'shit': 41, 'great': 42, 'probably': 43, 'everyone': 44, '2': 45, 'work': 46, 'take': 47, 'first': 48, 'actually': 49, 'use': 50, 'something': 51, 'let': 52, 'new': 53, 'day': 54, 'someone': 55, '3': 56, 'love': 57, 'team': 58, 'obviously': 59, 'point': 60, 'woman': 61, 'play': 62, 'bad': 63, 'fuck': 64, 'thought': 65, 'best': 66, 'maybe': 67, 'life': 68, 'give': 69, '1': 70, 'every': 71, 'pretty': 72, 'also': 73, 'ever': 74, 'must': 75, 'put': 76, 'white': 77, 'anything': 78, 'totally': 79, 'post': 80, 'real': 81, 'come': 8

In [48]:
len(t.word_index)

13647

In [49]:
X_train[0]

['black', 'shield', 'xpecial', 'confirmed', 'racist', 'insecure', 'manhood']

In [50]:
X_train = t.texts_to_sequences(X_train.tolist())

In [51]:
print(X_train[0])

[134, 382, 153]


In [52]:
X_test = t.texts_to_sequences(X_test)

In [53]:
len(X_train[200])

5

In [54]:
#Define maximum number of words to consider in each review
max_review_length = 300

In [55]:
#Pad training and test reviews
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train,
                                                        maxlen=max_review_length,
                                                        padding='pre',
                                                        truncating='post')
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, 
                                                       maxlen=max_review_length, 
                                                       padding='pre',truncating='post')

In [56]:
X_train.shape

(12000, 300)

In [57]:
X_test.shape

(3000, 300)

In [58]:
X_train[200]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

In [59]:
import gensim.downloader as api

In [60]:
#Load Glove model (similar to Word2Vec)
glove_model = api.load('glove-wiki-gigaword-50')

[==================================================] 100.0% 66.0/66.0MB downloaded


In [61]:
#Size of the model
glove_model.vectors.shape

(400000, 50)

In [62]:
#Embedding for word great
glove_model['and']

array([ 0.26818 ,  0.14346 , -0.27877 ,  0.016257,  0.11384 ,  0.69923 ,
       -0.51332 , -0.47368 , -0.33075 , -0.13834 ,  0.2702  ,  0.30938 ,
       -0.45012 , -0.4127  , -0.09932 ,  0.038085,  0.029749,  0.10076 ,
       -0.25058 , -0.51818 ,  0.34558 ,  0.44922 ,  0.48791 , -0.080866,
       -0.10121 , -1.3777  , -0.10866 , -0.23201 ,  0.012839, -0.46508 ,
        3.8463  ,  0.31362 ,  0.13643 , -0.52244 ,  0.3302  ,  0.33707 ,
       -0.35601 ,  0.32431 ,  0.12041 ,  0.3512  , -0.069043,  0.36885 ,
        0.25168 , -0.24517 ,  0.25381 ,  0.1367  , -0.31178 , -0.6321  ,
       -0.25028 , -0.38097 ], dtype=float32)

In [63]:
#Embedding length based on selected model - we are using 50d here.
embedding_vector_length = glove_model.vector_size

In [64]:
embedding_vector_length

50

In [65]:
#Initialize embedding matrix for our dataset with 10000+1 rows (1 for padding word)
#and 50 columns (as embedding size is 50)
embedding_matrix = np.zeros((desired_vocab_size + 1, embedding_vector_length))

In [66]:
embedding_matrix.shape

(1001, 50)

In [67]:
for word, i in sorted(t.word_index.items(),key=lambda x:x[1]):
    if i > (desired_vocab_size+1):
        break
    try:
        embedding_vector = glove_model[word] #Reading word's embedding from Glove model for a given word
        embedding_matrix[i] = embedding_vector
    except:
        pass

In [68]:
#Embedding for word 'movie' - index 17
embedding_matrix[17]

array([ 0.098481  , -0.10472   ,  0.42925   , -0.31915   ,  0.69646001,
       -0.42104   , -0.40799999,  0.44558999,  0.015993  ,  0.084546  ,
       -0.28307   ,  0.50590998, -0.46930999,  0.057326  ,  0.76236999,
        0.68702   ,  0.62682998, -0.05091   ,  0.43206999, -1.00259995,
       -0.45998999,  0.29291001,  0.45754001,  0.23255   ,  0.72465998,
       -1.79980004, -0.57512999,  0.38135001,  0.95319998, -0.80365998,
        2.91630006,  0.80574   , -0.48912001, -0.27897999, -0.14272   ,
       -0.055105  ,  0.21105   ,  0.50757003,  0.32238999, -0.61422002,
       -0.10563   , -0.036446  ,  0.1244    ,  0.50305003,  0.17504001,
        0.26899999, -0.14858   ,  0.30651999,  0.15998   ,  0.82944   ])

In [69]:
#Initialize model
tf.keras.backend.clear_session()
model = tf.keras.Sequential()

In [70]:
model.add(tf.keras.layers.Embedding(desired_vocab_size + 1, #Vocablury size
                                    embedding_vector_length, #Embedding size
                                    weights=[embedding_matrix], #Embeddings taken from pre-trained model
                                    trainable=False, #As embeddings are already available, we will not train this layer. It will act as lookup layer.
                                    input_length=max_review_length) #Number of words in each review
          )

In [71]:
model.output

<KerasTensor: shape=(None, 300, 50) dtype=float32 (created by layer 'embedding')>

In [72]:
#Flatten the data as we will use Dense layers
model.add(tf.keras.layers.Flatten())

In [73]:
model.output

<KerasTensor: shape=(None, 15000) dtype=float32 (created by layer 'flatten')>

In [74]:
#Add Hidden layers (Dense layers)
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(100, activation='relu', input_shape=()))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(50, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(25, activation='relu'))
model.add(tf.keras.layers.Dropout(0.25))

In [75]:
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [76]:
#Compile the model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [77]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 50)           50050     
                                                                 
 flatten (Flatten)           (None, 15000)             0         
                                                                 
 batch_normalization (BatchN  (None, 15000)            60000     
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 100)               1500100   
                                                                 
 batch_normalization_1 (Batc  (None, 100)              400       
 hNormalization)                                                 
                                                                 
 dense_1 (Dense)             (None, 50)                5

In [78]:
model.fit(X_train,y_train,
          epochs=5,
          batch_size=32,          
          validation_data=(X_test, y_test))

Epoch 1/5
375/375 [==============================] - 10s 24ms/step - loss: 0.7156 - accuracy: 0.5422 - val_loss: 0.6845 - val_accuracy: 0.5623
Epoch 2/5
375/375 [==============================] - 9s 25ms/step - loss: 0.6759 - accuracy: 0.5750 - val_loss: 0.6850 - val_accuracy: 0.5283
Epoch 3/5
375/375 [==============================] - 9s 24ms/step - loss: 0.6626 - accuracy: 0.6007 - val_loss: 0.6778 - val_accuracy: 0.5683
Epoch 4/5
375/375 [==============================] - 9s 24ms/step - loss: 0.6509 - accuracy: 0.6127 - val_loss: 0.6884 - val_accuracy: 0.5363
Epoch 5/5
375/375 [==============================] - 9s 24ms/step - loss: 0.6507 - accuracy: 0.6105 - val_loss: 0.6896 - val_accuracy: 0.5590


In [79]:
tf.keras.backend.clear_session()
model2 = tf.keras.Sequential()

In [80]:
model2.add(tf.keras.layers.Embedding(desired_vocab_size + 1, #Vocablury size
                                    embedding_vector_length, #Embedding size
                                    weights=[embedding_matrix], #Embeddings taken from pre-trained model
                                    trainable=False, #As embeddings are already available, we will not train this layer. It will act as lookup layer.
                                    input_length=max_review_length) #Number of words in each review
          )

In [81]:
model2.output

<KerasTensor: shape=(None, 300, 50) dtype=float32 (created by layer 'embedding')>

In [82]:
#Add first convolutional layer
model2.add(tf.keras.layers.Conv1D(32, #Number of filters 
                                 kernel_size=(3), #Size of the filter
                                 strides=1,
                                 activation='relu'))

#normalize data
model2.add(tf.keras.layers.BatchNormalization())

#Add second convolutional layer
model2.add(tf.keras.layers.Conv1D(64, kernel_size=(3), strides=2))
model2.add(tf.keras.layers.ReLU())

#normalize data
model2.add(tf.keras.layers.BatchNormalization())

In [83]:
model2.output

<KerasTensor: shape=(None, 148, 64) dtype=float32 (created by layer 'batch_normalization_1')>

In [84]:
#Use Global Average Pooling
model2.add(tf.keras.layers.GlobalAveragePooling1D())

model2.output

<KerasTensor: shape=(None, 64) dtype=float32 (created by layer 'global_average_pooling1d')>

In [85]:
#Output layer
model2.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [86]:
#Compile the model
model2.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [87]:
model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 50)           50050     
                                                                 
 conv1d (Conv1D)             (None, 298, 32)           4832      
                                                                 
 batch_normalization (BatchN  (None, 298, 32)          128       
 ormalization)                                                   
                                                                 
 conv1d_1 (Conv1D)           (None, 148, 64)           6208      
                                                                 
 re_lu (ReLU)                (None, 148, 64)           0         
                                                                 
 batch_normalization_1 (Batc  (None, 148, 64)          256       
 hNormalization)                                        

In [88]:
model2.fit(X_train,y_train,
          epochs=5,
          batch_size=32,          
          validation_data=(X_test, y_test))

Epoch 1/5
375/375 [==============================] - 12s 30ms/step - loss: 0.6800 - accuracy: 0.5605 - val_loss: 0.7183 - val_accuracy: 0.5020
Epoch 2/5
375/375 [==============================] - 12s 33ms/step - loss: 0.6670 - accuracy: 0.5915 - val_loss: 0.6692 - val_accuracy: 0.5810
Epoch 3/5
375/375 [==============================] - 11s 30ms/step - loss: 0.6568 - accuracy: 0.6058 - val_loss: 0.9452 - val_accuracy: 0.5070
Epoch 4/5
375/375 [==============================] - 11s 30ms/step - loss: 0.6480 - accuracy: 0.6178 - val_loss: 0.8948 - val_accuracy: 0.5143
Epoch 5/5
375/375 [==============================] - 11s 30ms/step - loss: 0.6377 - accuracy: 0.6258 - val_loss: 1.1548 - val_accuracy: 0.5100


In [89]:
model2.fit(X_train,y_train,
          epochs=5,
          batch_size=32,          
          validation_data=(X_test, y_test))

Epoch 1/5
375/375 [==============================] - 11s 30ms/step - loss: 0.6249 - accuracy: 0.6348 - val_loss: 0.6898 - val_accuracy: 0.5740
Epoch 2/5
375/375 [==============================] - 11s 30ms/step - loss: 0.6155 - accuracy: 0.6427 - val_loss: 2.3379 - val_accuracy: 0.4987
Epoch 3/5
375/375 [==============================] - 11s 30ms/step - loss: 0.6018 - accuracy: 0.6583 - val_loss: 2.6232 - val_accuracy: 0.5033
Epoch 4/5
375/375 [==============================] - 11s 30ms/step - loss: 0.5898 - accuracy: 0.6671 - val_loss: 2.4361 - val_accuracy: 0.5040
Epoch 5/5
375/375 [==============================] - 11s 30ms/step - loss: 0.5790 - accuracy: 0.6801 - val_loss: 1.9022 - val_accuracy: 0.5007


In [90]:
tf.keras.backend.clear_session()
model3 = tf.keras.Sequential()

In [91]:
model3.add(tf.keras.layers.Embedding(desired_vocab_size + 1, #Vocablury size
                                    embedding_vector_length, #Embedding size
                                    #weights=[embedding_matrix], #Embeddings taken from pre-trained model
                                    #trainable=False, #As embeddings are already available, we will not train this layer. It will act as lookup layer.
                                    input_length=max_review_length) #Number of words in each review
          )

In [92]:
#Add first convolutional layer
model3.add(tf.keras.layers.Conv1D(32, #Number of filters 
                                 kernel_size=(3), #Size of the filter
                                 strides=1,
                                 activation='relu'))

#normalize data
model3.add(tf.keras.layers.BatchNormalization())

#Add second convolutional layer
model3.add(tf.keras.layers.Conv1D(64, kernel_size=(3), strides=2))
model3.add(tf.keras.layers.ReLU())

#normalize data
model3.add(tf.keras.layers.BatchNormalization())

In [93]:
#Use Global Average Pooling
model3.add(tf.keras.layers.GlobalAveragePooling1D())

model3.output

<KerasTensor: shape=(None, 64) dtype=float32 (created by layer 'global_average_pooling1d')>

In [94]:
#Output layer
model3.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [95]:
#Compile the model
model3.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [96]:
model3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 50)           50050     
                                                                 
 conv1d (Conv1D)             (None, 298, 32)           4832      
                                                                 
 batch_normalization (BatchN  (None, 298, 32)          128       
 ormalization)                                                   
                                                                 
 conv1d_1 (Conv1D)           (None, 148, 64)           6208      
                                                                 
 re_lu (ReLU)                (None, 148, 64)           0         
                                                                 
 batch_normalization_1 (Batc  (None, 148, 64)          256       
 hNormalization)                                        

In [97]:
model3.fit(X_train,y_train,
          epochs=5,
          batch_size=32,          
          validation_data=(X_test, y_test))

Epoch 1/5
375/375 [==============================] - 15s 36ms/step - loss: 0.6734 - accuracy: 0.5753 - val_loss: 0.8998 - val_accuracy: 0.4977
Epoch 2/5
375/375 [==============================] - 15s 39ms/step - loss: 0.6285 - accuracy: 0.6352 - val_loss: 3.3955 - val_accuracy: 0.5023
Epoch 3/5
375/375 [==============================] - 13s 35ms/step - loss: 0.5823 - accuracy: 0.6747 - val_loss: 11.7522 - val_accuracy: 0.4977
Epoch 4/5
375/375 [==============================] - 13s 35ms/step - loss: 0.5240 - accuracy: 0.7122 - val_loss: 9.1274 - val_accuracy: 0.4977
Epoch 5/5
375/375 [==============================] - 14s 36ms/step - loss: 0.4720 - accuracy: 0.7383 - val_loss: 9.7716 - val_accuracy: 0.4977


In [98]:
model3.save('movie_review_cnn.h5')

In [99]:
model3.predict(X_test[0:1])

1/1 [==============================] - 0s 128ms/step


array([[1.6938781e-09]], dtype=float32)

In [100]:
y_test[0:1]

11499    0
Name: label, dtype: int64

In [101]:
from sklearn.metrics import accuracy_score, recall_score, classification_report, cohen_kappa_score
from sklearn import metrics 

In [108]:
####### Prediction ##########
predictions = model3.predict(X_test)

94/94 [==============================] - 1s 8ms/step


In [109]:
predictions1 = [0 if x < 0.5 else 1 for x in predictions]

In [110]:
print('Baseline: Accuracy: ', round(accuracy_score(y_test, predictions1)*100, 2))
print('\n Classification Report:\n', classification_report(y_test,predictions1))

Baseline: Accuracy:  49.77

 Classification Report:
               precision    recall  f1-score   support

           0       0.50      1.00      0.66      1493
           1       0.00      0.00      0.00      1507

    accuracy                           0.50      3000
   macro avg       0.25      0.50      0.33      3000
weighted avg       0.25      0.50      0.33      3000



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#